In [24]:
import requests
import pandas as pd

url = "https://www.forbes.com/billionaires/page-data/index/page-data.json"

response = requests.get(url)

json_data = response.json()

rows = json_data['result']['pageContext']['tableData']

ls_=[]

for row in rows:
    tableData = {
        k: v
        for (k, v) in row.items()
        if
        (k != 'person') & # dict
        (k != 'employment') & # dict
        (k != 'qas') & # two dicts
        (k != 'bios') & # list
        (k != 'abouts') & #list
        (k != 'csfDisplayFields') #list
        }
    ls_.append(tableData)

data = pd.DataFrame(ls_)

cols_to_drop = [
    'parentListUri',
    'organization',
    'title',
    'selfMadeRank',
    'residenceStateRegion',
    'embargo',
    'residenceMsa',
    'impactInvestor',
    'numberOfSiblings',
    'numberOfSiblingsEst',
    'bio',
    # 'totalCharitableDonation',
    # 'charitableDonationPercentage',
    'thumbnail',
    'notableDeal',
    'valueCreated',
    'primaryIndustry',
    'portraitImage',
    'landscapeImage',
    'clients']

data.drop(cols_to_drop, axis=1, inplace=True)

data['squareImage'].fillna('https://i.forbesimg.com/media/assets/forbes_1200x1200.jpg', inplace=True)

data['country'].fillna('Not Specified', inplace=True)

data['state'].fillna('Not Specified', inplace=True)

data['city'].fillna('Not Specified', inplace=True)

data['gender'].fillna('Not Specified', inplace=True)

data['birthDate'].fillna('Not Specified', inplace=True)

data['firstName'].fillna('Not Specified', inplace=True)

In [27]:
data_selected = data[[
    # 'name',
    # 'year',
    # 'month',
    # 'uri',
    'rank',
    # 'listUri',
    'finalWorth',
    'category',
    # 'otherCompensation',
    # 'visible',
    'personName',
    'age',
    'country',
    'state',
    'city',
    'source',
    'industries',
    'countryOfCitizenship',
    # 'timestamp',
    # 'version',
    # 'naturalId',
    'position',
    # 'imageExists',
    'selfMade',
    'status',
    'gender',
    # 'birthDate',
    'lastName',
    'firstName',
    # 'listDescription',
    # 'date',
    # 'wealthList',
    # 'familyList',
    # 'squareImage',
    # 'bioSuppress',
    # 'suppressOnProfiles',
    # 'premiumProfile',
    # 'philanthropyScore',
    'netWorth'
]]

In [29]:
data_selected.head()

,rank,finalWorth,category,personName,age,country,state,city,source,industries,countryOfCitizenship,position,selfMade,status,gender,lastName,firstName,netWorth
0,1,219000,Automotive,Elon Musk,50,United States,Texas,Austin,"Tesla, SpaceX",Automotive,United States,1,True,U,M,Musk,Elon,$219 B
1,2,171000,Technology,Jeff Bezos,58,United States,Washington,Seattle,Amazon,Technology,United States,2,True,D,M,Bezos,Jeff,$171 B
2,3,158000,Fashion & Retail,Bernard Arnault & family,73,France,Not Specified,Paris,LVMH,Fashion & Retail,France,3,False,U,M,Arnault,Bernard,$158 B
3,4,129000,Technology,Bill Gates,66,United States,Washington,Medina,Microsoft,Technology,United States,4,True,U,M,Gates,Bill,$129 B
4,5,118000,Finance & Investments,Warren Buffett,91,United States,Nebraska,Omaha,Berkshire Hathaway,Finance & Investments,United States,5,True,U,M,Buffett,Warren,$118 B


In [36]:
def convert_networth(value):
    abbreviations = {"B": 1000000000, "M": 1000000, "K": 1000}
    value = value.replace("$", "")
    for abbr, factor in abbreviations.items():
        if abbr in value:
            return float(value.replace(abbr, "")) * factor
    return float(value)

data_selected['netWorth'] = data_selected['netWorth'].apply(convert_networth)


<ipython-input-36-6d521c8c011b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['netWorth'] = data_selected['netWorth'].apply(convert_networth)


In [37]:
correlation = data_selected[['age', 'netWorth']].corr().loc['age', 'netWorth']
correlation

0.04429162120019725

In [38]:
data_selected.head()

,rank,finalWorth,category,personName,age,country,state,city,source,industries,countryOfCitizenship,position,selfMade,status,gender,lastName,firstName,netWorth
0,1,219000,Automotive,Elon Musk,50,United States,Texas,Austin,"Tesla, SpaceX",Automotive,United States,1,True,U,M,Musk,Elon,2.190000e+11
1,2,171000,Technology,Jeff Bezos,58,United States,Washington,Seattle,Amazon,Technology,United States,2,True,D,M,Bezos,Jeff,1.710000e+11
2,3,158000,Fashion & Retail,Bernard Arnault & family,73,France,Not Specified,Paris,LVMH,Fashion & Retail,France,3,False,U,M,Arnault,Bernard,1.580000e+11
3,4,129000,Technology,Bill Gates,66,United States,Washington,Medina,Microsoft,Technology,United States,4,True,U,M,Gates,Bill,1.290000e+11
4,5,118000,Finance & Investments,Warren Buffett,91,United States,Nebraska,Omaha,Berkshire Hathaway,Finance & Investments,United States,5,True,U,M,Buffett,Warren,1.180000e+11
